In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json
from requests.auth import HTTPBasicAuth
from flatten_json import flatten

In [2]:
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

apikey = "RGAPI-94ae027f-6d09-4105-ae83-ec6c03421c05"

In [3]:
# get challenger summonerId from league-exp-api
# 1.2 delay bc short list

req = requests.get("https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1&api_key={}"
                   .format(apikey),time.sleep(1.2))
assert req.status_code == 200
data = req.json()
sumids = [d['summonerId'] for d in data]

In [4]:
print(sumids[0])

3iHv5pGVlyyFUFpy9aHa76M2zrHeYHV0fzrTY7omUNTgOmg


In [5]:
# get accountId via summonerId via summoner-api
# 1.2 delay bc short list

accids = []
for sumid in sumids:
    sumreq = requests.get("https://kr.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}"
                          .format(sumid,apikey),time.sleep(1.2))
    assert sumreq.status_code == 200
    accids.append(sumreq.json()['accountId'])

In [6]:
print(accids[0])

fMzg3xXcmAo_CI6ACXOA7OvRm3CB20jiJi4nUbjEC2Ks8tk


In [7]:
# get game ids from match history via match-api
# 1.3 delay bc long list

gamids = []

for acc in accids:
    matlreq = requests.get("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.3))
    assert matlreq.status_code == 200
    gamids += [d['gameId'] for d in matlreq.json()['matches']]

In [17]:
# print original length, then correct overlap from shared games and print new length

print(len(gamids))

gamids = list(set(gamids))

print(len(gamids))

9844
9844


In [9]:
with open('kr_chall_gam_id_list.txt', 'w') as f:
    for item in gamids:
        f.write("%s\n" % item)

In [ ]:
#with open('kr_chall_gam_id_list1.txt', 'r') as f:
#    finchallgames = f.read().splitlines()

In [ ]:
#finchallgames =set([int(e) for e in finchallgames])

In [ ]:
#len(gamids - (finchallgames & gamids))

In [ ]:
#gamids = gamids - finchallgames

In [18]:
# get individual match information from gamids list via match-api
# 1.5 delay bc long list and repeated issues with 

matchlist = []

for gam in gamids[8788:]:
    gamreq = requests.get("https://kr.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
    
    i=0     
    while gamreq.status_code != 200:
        gamreq = requests.get("https://kr.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            assert gamreq.status_code == 200
            
    match = flatten(gamreq.json())
    matchlist.append(match)

In [12]:
gamreq.status_code

200

In [13]:
# check length of matchlist 

check = [len(m) for m in matchlist]
len(matchlist),max(check),min(check)

(8788, 1550, 1192)

In [14]:
# create df from matches

#df = pd.DataFrame(matchlist)

In [15]:
df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,teams_1_bans,participants_0_timeline_csDiffPerMinDeltas_30-end,participants_0_timeline_xpDiffPerMinDeltas_30-end,participants_0_timeline_damageTakenDiffPerMinDeltas_30-end,participants_3_timeline_csDiffPerMinDeltas_30-end,participants_3_timeline_xpDiffPerMinDeltas_30-end,participants_3_timeline_damageTakenDiffPerMinDeltas_30-end,participants_7_timeline_csDiffPerMinDeltas_30-end,participants_7_timeline_xpDiffPerMinDeltas_30-end,participants_7_timeline_damageTakenDiffPerMinDeltas_30-end
0,4423090177,KR,1591086309502,751,900,11,13,10.11.322.2991,URF,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4536860675,KR,1595927443677,2106,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4536991748,KR,1595932292463,1992,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4478861319,KR,1593442472883,1899,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4530798600,KR,1595686706534,607,1300,21,13,10.15.328.8759,NEXUSBLITZ,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8783,4526305609,KR,1595530941906,1171,1300,21,13,10.15.328.8759,NEXUSBLITZ,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8784,4493537612,KR,1594098982141,1622,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8785,4506579283,KR,1594645888668,944,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8786,4542099797,KR,1596130864320,1446,420,11,13,10.15.330.344,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df1 = pd.DataFrame(matchlist)

In [21]:
df = df.append(df1)

In [22]:
compression_opts = dict(method='zip',
                        archive_name='krchall.csv')  
df.to_csv('krchall.zip', index=False,
          compression=compression_opts)